In [2]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import hail as hl
hl.init() 

Running on Apache Spark version 2.4.1
SparkUI available at http://633fc3f91d0f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.29-cf730c8fc8f6
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191220-1548-0.2.29-cf730c8fc8f6.log


In [5]:
cd = hl.import_table('./cadd/whole_genome_SNVs.tsv.gz', force_bgz = True, delimiter='\t', comment='#', 
                     types = {'f0':'tstr', 'f1':'tint', 'f2':'tstr', 'f3':'tstr', 'f4':'tfloat', 'f5':'tfloat'},
                     no_header = True)

2019-12-20 15:48:45 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'str' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'float64' (user-specified)
  Loading column 'f5' as type 'float64' (user-specified)



In [6]:
cd = cd.select(variants = hl.parse_variant('chr'+hl.str(cd.f0)+':'+hl.str(cd.f1)+':'+hl.str(cd.f2)+':'+hl.str(cd.f3), reference_genome='GRCh38'),
               cadd_score = cd.f5)

In [7]:
cd = cd.key_by(cd.variants.locus,cd.variants.alleles)

In [8]:
mt = hl.read_matrix_table('step3_output.mt')

In [9]:
mt.row_key

<StructExpression of type struct{locus: locus<GRCh38>, alleles: array<str>}>

In [10]:
cd.key

<StructExpression of type struct{locus: locus<GRCh38>, alleles: array<str>}>

In [11]:
mt = mt.annotate_rows(cadd = cd[mt.row_key])

In [12]:
mt = mt.transmute_rows(cadd = mt.cadd.cadd_score)

In [13]:
mt.checkpoint('step4_output.mt')

2019-12-20 17:31:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 01:35:43 Hail: INFO: wrote matrix table with 7615409 rows and 151 columns in 591 partitions to step4_output.mt
